In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
import pickle
from sklearn.model_selection import KFold


In [3]:
transactions    = pd.read_csv('sales_train.csv.gz')
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
test            = pd.read_csv('test.csv.gz')

In [59]:
transactions = pd.merge(transactions, items, on='item_id', how='left')
transactions = transactions.drop('item_name', axis=1)
transactions = transactions[transactions['date_block_num'] > 12]
transactions.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id_x,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean,item_category_id_y,item_category_id
0,13,27,15242,699.0,1.0,63,2.0,8.0,1.000000,666.750000,4208.0,1.298365,952.135647,1281.0,1.051724,780.414491,63,63
1,13,27,15200,299.0,1.0,69,1.0,1.0,1.000000,299.000000,4208.0,1.298365,952.135647,887.0,1.251058,396.144100,69,69
2,13,27,15279,799.0,1.0,63,2.0,48.0,1.043478,799.000000,4208.0,1.298365,952.135647,1281.0,1.051724,780.414491,63,63
3,13,27,15202,299.0,1.0,69,1.0,3.0,1.000000,365.666667,4208.0,1.298365,952.135647,887.0,1.251058,396.144100,69,69
4,13,27,14888,549.0,1.0,55,1.0,19.0,1.000000,545.526316,4208.0,1.298365,952.135647,9442.0,1.018005,286.466544,55,55


In [5]:
train_item_ids = transactions['item_id'].unique()
train_shop_ids = transactions['shop_id'].unique()
train_blocks = transactions['date_block_num'].unique()

In [71]:
combinations = []
for shop_id in train_shop_ids:
    #get all item ever sold by this shop
    sold_items = transactions[transactions['shop_id'] == shop_id]['item_id'].unique()
    shop_combos = [[item, shop_id, block] for item in sold_items for block in train_blocks]
    for shop_combo in shop_combos:
        combinations.append(shop_combo)

In [73]:
combinations_test = []
for shop_id in test['shop_id'].unique():
    #get all item ever sold by this shop
    sold_items = test[test['shop_id'] == shop_id]['item_id'].unique()
    shop_combos = [[item, shop_id, block] for item in sold_items for block in train_blocks]
    for shop_combo in shop_combos:
        combinations_test.append(shop_combo)

In [75]:
all_combos = pd.DataFrame(np.unique(np.vstack([combinations, combinations_test]), axis=0), columns=['item_id', 'shop_id', 'date_block_num'])

In [76]:
len(all_combos)

7722666

In [77]:
all_combos.head()

,item_id,shop_id,date_block_num
0,0,54,13
1,0,54,14
2,0,54,15
3,0,54,16
4,0,54,17


In [61]:
transactions['y'] = transactions.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].transform(np.sum)

In [102]:
gc.collect()

transactions['item_cnt_sum'] = transactions.groupby(['item_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['item_cnt_mean'] = transactions.groupby(['item_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['item_price_mean'] = transactions.groupby(['item_id', 'date_block_num'])['item_price'].transform(np.mean)

transactions['shop_cnt_sum'] = transactions.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['shop_cnt_mean'] = transactions.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['shop_price_mean'] = transactions.groupby(['shop_id', 'date_block_num'])['item_price'].transform(np.mean)

transactions['category_cnt_sum'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_cnt_day'].transform(np.sum)
transactions['category_cnt_mean'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_cnt_day'].transform(np.mean)
transactions['category_price_mean'] = transactions.groupby(['item_category_id', 'date_block_num'])['item_price'].transform(np.mean)

training = pd.merge(all_combos,transactions,on=['item_id', 'shop_id', 'date_block_num'],how='left').fillna(0)

In [103]:
training = training.sample(frac=1)

In [104]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

training_int = training.select_dtypes(include=['int'])
converted_int = training_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(training_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([training_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

257.07 MB
96.40 MB


,before,after
uint8,NaN,2.0
uint16,NaN,1.0
int64,3.0,NaN


In [105]:
training_float = training.select_dtypes(include=['float'])
converted_float = training_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(training_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([training_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

1028.26 MB
546.26 MB


,before,after
float32,NaN,15.0
float64,15.0,NaN


In [106]:
optimized_training = training.copy()

optimized_training[converted_int.columns] = converted_int
optimized_training[converted_float.columns] = converted_float

print(mem_usage(training))
print(mem_usage(optimized_training))

1221.06 MB
578.40 MB


In [107]:
training = optimized_training
del optimized_training
gc.collect()

56

In [123]:
import pickle as pickle

#pickle.dump(training, open( "training", "wb"))

training = pickle.load( open( "training", "rb" ) )

In [98]:
len(training)

8423524

In [99]:
lag_columns = training.columns[7:]
lags = [1,2,3,6,12]

In [127]:
lagged.head()

,lagged_block,shop_id,item_id,item_price,item_cnt_day,item_category_id_x,y,item_cnt_sum_lag_1,item_cnt_mean_lag_1,item_price_mean_lag_1,shop_cnt_sum_lag_1,shop_cnt_mean_lag_1,shop_price_mean_lag_1,category_cnt_sum_lag_1,category_cnt_mean_lag_1,category_price_mean_lag_1,item_category_id_y_lag_1,item_category_id_lag_1
0,13,27,15242,699.0,1.0,63,2.0,8.0,1.000000,666.750000,4208.0,1.298365,952.135647,1281.0,1.051724,780.414491,63,63
1,13,27,15200,299.0,1.0,69,1.0,1.0,1.000000,299.000000,4208.0,1.298365,952.135647,887.0,1.251058,396.144100,69,69
2,13,27,15279,799.0,1.0,63,2.0,48.0,1.043478,799.000000,4208.0,1.298365,952.135647,1281.0,1.051724,780.414491,63,63
3,13,27,15202,299.0,1.0,69,1.0,3.0,1.000000,365.666667,4208.0,1.298365,952.135647,887.0,1.251058,396.144100,69,69
4,13,27,14888,549.0,1.0,55,1.0,19.0,1.000000,545.526316,4208.0,1.298365,952.135647,9442.0,1.018005,286.466544,55,55


In [126]:
training.head()

,item_id,shop_id,date_block_num,item_price,item_cnt_day,item_category_id_x,y,item_cnt_sum,item_cnt_mean,item_price_mean,shop_cnt_sum,shop_cnt_mean,shop_price_mean,category_cnt_sum,category_cnt_mean,category_price_mean,item_category_id_y,item_category_id,lagged_block
225663,19717,57,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
1920455,20024,14,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22
5102479,7322,41,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27
563780,528,21,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26
6281789,18110,54,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18


In [ ]:
gc.collect()
def lagged_name(lag_column, lag):
    return "%s_lag_%d" % (lag_column, lag)

merge_columns = ['lagged_block','item_id','shop_id']

for lag in lags:
    lagged = transactions.copy()
    lagged.rename(columns={'date_block_num':'lagged_block'},inplace=True)
    training['lagged_block'] = training['date_block_num'] - lag
    lagged_names = [lagged_name(c,lag) for c in lag_columns]
    lag_mapping = dict(zip(lag_columns, lagged_names))
    lagged.rename(columns=lag_mapping,inplace=True)
    training = pd.merge(training,lagged[lagged_names+merge_columns],on=merge_columns)
    del lagged
    gc.collect()

In [131]:
[1,2,3] + [4]

[1, 2, 3, 4]